# Scikit-Learn IRIS Model

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on a kubernetes cluster
 
## Dependencies

 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:8080`
 

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


Create Seldon Core config file

In [2]:
%%writefile sklearn_iris_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.3
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

Overwriting sklearn_iris_deployment.yaml


In [3]:
!kubectl create -f sklearn_iris_deployment.yaml -n seldon

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [4]:
!kubectl wait sdep/seldon-deployment-example \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/seldon-deployment-example condition met


In [5]:
import json

from tenacity import retry, stop_after_delay, wait_exponential


@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    res = !curl  -s http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'
    j = json.loads(res[0])
    assert j["data"]["ndarray"][0][0] > 0.0


predict()

REST request with raw_data

In [6]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="seldon-deployment-example", namespace="seldon")

2025-11-18 16:32:29.147225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763483549.164286 3652725 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763483549.169565 3652725 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-18 16:32:29.187920: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    res = sc.predict(
        gateway="istio",
        gateway_endpoint="localhost:8003",
        transport="rest",
        raw_data={"data": {"ndarray": [[5.964, 4.006, 2.081, 1.031]]}},
    )
    print(res.response)
    assert res.success == True


predict()

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.9548873249364059, 0.04505474761562512, 5.7927447968953825e-05]]}, 'meta': {'requestPath': {'sklearn-iris-classifier': 'seldonio/sklearn-iris:0.3'}}}


gRCP request with proto raw_data

In [8]:
from seldon_core.utils import json_to_seldon_message


@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    proto_raw_data = json_to_seldon_message(
        {"data": {"ndarray": [[5.964, 4.006, 2.081, 1.031]]}}
    )
    res = sc.predict(
        gateway="istio",
        gateway_endpoint="localhost:8003",
        transport="grpc",
        raw_data=proto_raw_data,
    )
    print(res)
    assert res.success == True


predict()

Success:True message:
Request:
{'data': {'ndarray': [[5.964, 4.006, 2.081, 1.031]]}}
Response:
{'meta': {'requestPath': {'sklearn-iris-classifier': 'seldonio/sklearn-iris:0.3'}}, 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.9548873249364059, 0.04505474761562512, 5.7927447968953825e-05]]}}


In [9]:
!kubectl delete -f sklearn_iris_deployment.yaml -n seldon

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted
